# Motion estimation

In this section we will go through how to perform motion analysis on MPS data. The material here is mostly based on the documentation at https://computationalphysiology.github.io/mps_motion

In [ ]:
import mps
import mps_motion

In [ ]:
motion_data = "motion_data.npy"
mps_motion.utils.download_demo_data(motion_data)

In [ ]:
data = mps.MPS(motion_data)